In [1]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import cluster, metrics

file = open("../log/n3 (1).pickle", "rb")
data = pickle.load(file)
file.close()
scene_info = data['ml']['scene_info']
scene_command = data['ml']['command']

for i in range(2, 3000):
    path = "../log/n3 (" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    scene_info = scene_info + data['ml']['scene_info']
    scene_command = scene_command + data['ml']['command']
    file.close()
    
print(len(scene_info))
print(len(scene_command))

9734722
9734722


In [2]:
# 提取特徵

scene_info = data['ml']['scene_info']
scene_command = data['ml']['command']

Ball_x = []
Ball_y = []
Ball_speed_x = []
Ball_speed_y = []
Direction = []
Platform = []
Command = []

for i, s in enumerate(scene_info[1:-2]):
    Ball_x.append(s['ball'][0])
    Ball_y.append(s['ball'][1])
    Platform.append(s['platform'][0])
    Ball_speed_x.append(scene_info[i+2]["ball"][0] - scene_info[i+1]["ball"][0])
    Ball_speed_y.append(scene_info[i+2]["ball"][1] - scene_info[i+1]["ball"][1])
    if Ball_speed_x[-1] > 0:
        if Ball_speed_y[-1] > 0:
            # 右下
            Direction.append(0)
        else:
            # 右上
            Direction.append(1)
    else:
        if Ball_speed_y[-1] > 0:
            # 左下
            Direction.append(2)
        else:
            # 左上
            Direction.append(3)
            
for c in scene_command[1:-2]:
    if c == "NONE":
        Command.append(0)
    elif c == "MOVE_LEFT":
        Command.append(-1)
    elif c == "MOVE_RIGHT":
        Command.append(1)


feature = np.array([Ball_x, Ball_y, Ball_speed_x, Ball_speed_y, Direction, Platform])
X = np.transpose(feature) 
y = scene_command

print(feature)

print(scene_command[1])
scene_command[1] = 0

[[ 93 100 107 ... 105 112 119]
 [395 388 381 ...  56  63  70]
 [  7   7   7 ...   7   7   7]
 [ -7  -7  -7 ...   7   7   5]
 [  1   1   1 ...   0   0   0]
 [ 75  70  65 ...  45  45  40]]
MOVE_LEFT


In [3]:
# 提取特徵
import numpy as np

scene_info = data['ml']['scene_info']
sene_command = data['ml']['command']

k = range(1, len(scene_info)-1)

ball_x = np.array([scene_info[i]['ball'][0] for i in k])
ball_y = np.array([scene_info[i]['ball'][1] for i in k])
ball_speed_x = np.array([scene_info[i+1]['ball'][0] - scene_info[i]['ball'][0] for i in k])
ball_speed_y = np.array([scene_info[i+1]['ball'][1] - scene_info[i]['ball'][1] for i in k])
direction = np.where(np.vstack((ball_speed_x, ball_speed_y)) > 0, [[1],[0]], [[2],[3]]).sum(axis=0)  # x y: ++1, +-4, -+2, --3
platform = np.array([scene_info[i]['platform'][0] for i in k])
target = np.where(np.array(scene_command) == 'NONE', 0,
                  np.where(np.array(scene_command) == 'MOVE_LEFT', -1, 1))[1:-1]  # [0] SERVE_TO_RIGHT, [1897] None


In [4]:
for i in range(2, len(scene_info) - 1):
    g = scene_info[i]
    feature = np.hstack((ball_x.reshape(-1, 1),
               ball_y.reshape(-1, 1),
               ball_speed_x.reshape(-1, 1),
               ball_speed_y.reshape(-1, 1),
               direction.reshape(-1, 1),
               platform.reshape(-1, 1)))
    if scene_command[i] == "NONE": scene_command[i] = 0
    elif scene_command[i] == "MOVE_LEFT": scene_command[i] = 1
    else: scene_command[i] = 2
    
answer = np.array(scene_command[1:-1])

print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

mask = [5, 2, 3, 4]
X = feature[:, mask]

[[ 93 395   7  -7   4  75]
 [100 388   7  -7   4  70]
 [107 381   7  -7   4  65]
 ...
 [112  63   7   7   1  45]
 [119  70   7   5   1  40]
 [126  75   4  -7   4  35]]
(2091, 6)
[0 1 0 ... 1 1 0]
(2091,)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, answer, test_size=0.2)
knn = KNeighborsClassifier(n_neighbors = 4)
platform_predict_clf = knn.fit(x_train, y_train)

y_predict = platform_predict_clf.predict(x_test)
print(y_predict)

accuracy = metrics.accuracy_score(y_test, y_predict)
print("Accuracy = {:8.3f}%".format(accuracy*100))

[2 0 2 1 1 0 1 2 2 1 1 0 1 0 0 1 1 2 1 1 1 1 1 2 1 1 1 1 0 0 1 1 2 0 1 0 1
 1 2 0 0 0 0 0 0 1 2 1 0 0 1 0 2 1 1 0 0 0 2 1 1 2 1 1 2 0 2 1 0 0 0 0 2 1
 0 0 0 1 2 0 1 2 0 1 0 2 0 0 1 2 0 1 1 1 1 2 1 0 1 1 0 2 0 0 1 0 2 2 1 2 0
 1 1 1 2 1 1 1 1 2 1 2 2 1 0 1 1 1 0 2 1 1 1 1 1 1 2 0 1 2 1 1 2 1 2 1 0 2
 0 0 0 0 0 0 1 2 1 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 1 2 0 2 1 2 1 0
 0 1 1 2 1 1 1 1 1 1 1 1 1 0 2 0 0 2 0 2 0 1 1 0 0 1 2 2 0 0 2 1 0 0 0 1 1
 2 1 1 0 2 0 1 1 0 0 0 1 2 0 1 1 0 1 2 1 1 2 1 1 0 2 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 0 1 1 1 0 1 1 0 1 0 2 2 0 0 0 1 2 1 0 0 1 1 1 2 1 2 0 0 1 2 1 0 2 0
 1 1 0 1 0 0 1 2 0 0 2 0 0 1 1 1 2 1 0 0 1 1 1 1 2 1 1 1 1 2 1 2 2 1 0 2 0
 1 1 1 1 1 0 0 2 1 1 2 2 1 0 2 1 0 0 1 0 0 2 1 1 1 1 1 0 1 2 2 0 2 2 0 1 0
 2 1 0 2 2 2 0 2 1 0 0 1 1 1 2 0 0 0 1 2 1 1 1 1 1 0 1 0 0 2 2 1 0 0 1 1 1
 1 1 2 1 0 1 1 2 2 1 1 0]
Accuracy =   46.778%


In [6]:
file = open('model.pickle', 'wb')
pickle.dump(knn, file)
file.close()